In [2]:
# imports
from composable import pipeable
import re as r


In [3]:
# reading in file
with open('sell_short_trades_copy.txt') as f:
    content = f.readlines()
example = content[:29]
example = [line.strip() for line in example]
example

['',
 'DOTC                                                                                 RUN DATE : 06/14/17 21:56',
 'RPT DATE :06/14/17',
 'TTS0126:SELL SHORT TRADES & SHORT EXEMPT',
 'Symbol   Side    Cxl      Qty    Price       Bid        Ask        T-DatS-DatTradeID      TradeTiSS      Exbkr',
 '',
 'TradeCommType  SourceCommission   Account   OrderID      GTL               Trailer Info               Clr',
 'CERS     SS      NEW        2,756   2.400000    2.340000   2.45000006/1406/191706149900003 09:30:CustSS',
 '/17  /17  256945       00',
 '0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=GSCO',
 'CERS     SS      NEW          100   2.36000018422.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX',
 '/17  /17  866087       04',
 '0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS',
 '854300',
 'CERS     SS      NEW          200   2.350000    2.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX',
 '/17  /17  866096       04',
 '0

In [4]:
line_start = r.compile(r'(CERS    |S )')
re_split_first_line_test = [line.strip() for line in example if line_start.match(line)]
re_split_first_line_test[:20]
# i thought maybe I test the beginning of each line with a regex

['CERS     SS      NEW        2,756   2.400000    2.340000   2.45000006/1406/191706149900003 09:30:CustSS',
 'CERS     SS      NEW          100   2.36000018422.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX',
 'CERS     SS      NEW          200   2.350000    2.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX',
 'CERS     SS      NEW          100   2.350000    2.350000   2.36000006/1406/191706149900003 10:20:ContraSSFREX',
 'CERS     SS      NEW          100   2.350000    2.350000   2.36000006/1406/191706149900003 10:22:ContraSSFREX',
 'CERS     SS      NEW          100   2.360000    2.350000   2.36000006/1406/191706149900003 10:24:ContraSSFREX']

In [5]:
re_split_first_line = [line.strip() for line in content if line_start.match(line)]
re_split_first_line[:20]
# this worked, but now i have lost each row in between 
# I decided to give up on regex style

[]

In [6]:
# all my lambdas to check for what to get rid of
# after all these functions, I will only have the rows of data. No headers or extra lines
is_blank = lambda l: True if len(l.strip()) == 0 else False # blank lines or lines filled with only whitespace

# removal with regular expressions (I know I have to remove multiple of these types of lines)
dr = r.compile(r"\s*\d+ of 77\s*") # regex for page number lines
is_page_num_line = pipeable(lambda l: True if dr.match(l) else False) # check for page number lines
headers = r.compile(r'^( Symbol | TradeCommType ).*') # regex for header lines
no_headers = pipeable (lambda l: True if headers.match(l) else False) # check for header lines

# basically copied and pasted these lambdas and changed what the startswith() paramater was, because these lines were specific and this was pretty quick
# I only have to remove one of each of these types of lines
totals = pipeable(lambda l: True if l.strip().startswith('TOTAL TRADES:') else False) # check for TOTAL TRADES line
end = pipeable(lambda l: True if l.strip().startswith('*****') else False) # check for END OF REPORT line
header1 = pipeable(lambda l: True if l.strip().startswith('DOTC') else False) # check for first header line
header2 = pipeable(lambda l: True if l.strip().startswith('RPT DATE') else False) # check for second header line
header3 = pipeable(lambda l: True if l.strip().startswith('TTS0126:SELL') else False) # check for third header line

needs_removal = pipeable(lambda l: is_blank(l) or is_page_num_line(l) or no_headers(l) or totals(l) or end(l) or header1(l) or header2(l) or header3(l)) # full removal lambda

In [7]:
pure_data_lines = [l.strip() for l in content if not needs_removal(l)] 
len(pure_data_lines) # check to see if it is reasonable length
# original data set length was 

3499

In [8]:
example1 = example[7]
if line_start.match(example1):
    print('yay')


yay


In [9]:
example1 = example[8]
if line_start.match(example1):
    print('yay')

In [80]:
# Back to regex stuff... Is there a way to add lines if they match/don't match? Lets find out
testingdata1 = pure_data_lines[:20]
line_start2 = r.compile(r'^((?!(CERS    |S    )).)*$')

def regex_combine(data):
    """
    """
    lis = list(data)
    current = 0
    next = 1

    while(current < len(lis) and (starts_condition(lis[current]) == False)): # get rid of all the lines in front of the ones that will begin with 'CERS ' or 'S '
        # print('bye')
        del(lis[current])

    while(current < len(lis)): # updating the len(list) as we go so we do not go out of range with our index of current
        current_item = lis[current] # get current item
        next = current + 1 # next item will start as 1 after current
        if (next >= len(lis)): # check that next is not out of range index
            break
        next_item = lis[next] # get next item
        if line_start.match(current_item): # make sure current item fulfills start condition
            while line_start2.match(next_item): # make sure next item fulfills its start condition
                # print(next_item)
                lis[current] = lis[current] + ' ' + next_item # add the two together

                next += 1 # prepare to get the next item over
                if (next >= len(lis)): # check that next item is not out of range index
                    break
                next_item = lis[next] # get next item

            del lis[current+1:next] # delete the 'next' items that we added to current

        current += 1 # increment

    return lis

    # I kept messing around with this trying to make it work
    # but i can't do (while not line_start.match(next_item)): because it returns None, not True/False...
    # BUT!!!! i did find a way to use r.compile(r'^((?!(CERS    |S )).)*$') to match rows that do not contain the CERS or S (thanks stack overflow)

In [11]:
line_start2 = r.compile(r'^((?!(CERS     |S    )).)*$')
re_split_first_line_test2 = [line.strip() for line in example if line_start2.match(line)]
re_split_first_line_test2[:20]

['',
 'DOTC                                                                                 RUN DATE : 06/14/17 21:56',
 'RPT DATE :06/14/17',
 'TTS0126:SELL SHORT TRADES & SHORT EXEMPT',
 '',
 'TradeCommType  SourceCommission   Account   OrderID      GTL               Trailer Info               Clr',
 '/17  /17  256945       00',
 '0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=GSCO',
 '/17  /17  866087       04',
 '0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS',
 '854300',
 '/17  /17  866096       04',
 '0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS',
 '854300',
 '/17  /17  869780       23',
 '0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS',
 '854300',
 '/17  /17  892888       29',
 '0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS',
 '854300']

In [66]:
regex_combine(testingdata1) # example test

['CERS     SS      NEW        2,756   2.400000    2.340000   2.45000006/1406/191706149900003 09:30:CustSS /17  /17  256945       00 0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=GSCO',
 'CERS     SS      NEW          100   2.36000018422.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX /17  /17  866087       04 0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS 854300',
 'CERS     SS      NEW          200   2.350000    2.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX /17  /17  866096       04 0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS 854300',
 'CERS     SS      NEW          100   2.350000    2.350000   2.36000006/1406/191706149900003 10:20:ContraSSFREX /17  /17  869780       23 0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS 854300',
 'CERS     SS      NEW          100   2.350000    2.350000   2.36000006/1406/191706149900003 10:22:ContraSSFREX /17  /17  892888       29 0704 PerUnit   TERM

In [67]:
sorted = regex_combine(pure_data_lines) # tried it on full data set
# BOOOM baby
len(sorted)

913

In [68]:
# now I will try it without regex because its fun
starts_condition = pipeable(lambda line: True if line.startswith('CERS ') or line.startswith('S ') else False)

def combine_rows(data):
    """
    takes a list of strings whose data is split over multiple lines and combines the right rows into one line for each
    Args: list of strings
    returns: lists with each row of strings on one line
    """
    data = list(data) # establish that our data must be in a list if it isn't already
    counter = 0 # current item counter
    next = 0 # next item counter

    while (counter < len(data) and (starts_condition(data[counter]) == False)): # get rid of all the lines in front of the ones that will begin with 'CERS ' or 'S '
        # print('bye')
        del(data[counter])

    stop_condition = len(data) # stop condition for loop is the length of list

    while (counter < stop_condition): # while we are still within the length of the list. This will stop once our current item counter is at the end of the list
        
        current_item = data[counter] # get current item (first item will be data[0])
        # print(current_item)
        
        if next >= len(data): # check for out of bounds (we are at the end of the list)
            break
        else:
            next = counter + 1 # make the next counter 1 greater than the current counter value
            next_item = data[next] # get the next item in the list

        if (starts_condition(current_item) == True): # if the current item fulfills start condition, we want to begin the loop of adding the following lines
            while not (starts_condition(next_item)): # while the next line does not start with 'CERS' or 'S' or 'STOP'
                data[counter] = data[counter] + " " + next_item # adds the next line to the previous
                # print(data[counter])
                
                next += 1 # increment next counter
                if next >= len(data): # without this, when we get to the end of the list, it will index out of range because of the increment
                    break
                next_item = data[next] # get next item in list

            del data[counter+1:next] # delete the items that we just added to current item
            stop_condition = len(data) # update stop condition to the length of our updated list

        counter += 1 # increment current item counter

    return data

    # I got this one to work too, but using the string.startswith() function, but I will test with my regex one instead 

In [85]:
def test_regex_combine():
    assert regex_combine(['CERS    34', 'asdf jkl', 'S    444', 'H', 'E', 'L', 'L', 'O', 'CERS    ']) == ['CERS    34 asdf jkl', 'S    444 H E L L O', 'CERS    ']
    assert regex_combine(['sefe', 'sdfs']) == []
    assert regex_combine(['SS', 'CERS    dsf', 'dsf', 'dsf', 'S    ', 'SS']) == ['CERS    dsf dsf dsf', 'S     SS']
    assert regex_combine(['CERS     SS      NEW        2,756   2.400000    2.340000   2.45000006/1406/191706149900003 09:30:CustSS', '/17  /17  256945       00', '0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=GSCO', 'CERS     SS      NEW          100   2.36000018422.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX', '/17  /17  866087       04', '0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS']) == ['CERS     SS      NEW        2,756   2.400000    2.340000   2.45000006/1406/191706149900003 09:30:CustSS /17  /17  256945       00 0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=GSCO',
 'CERS     SS      NEW          100   2.36000018422.360000   2.37000006/1406/191706149900003 10:20:ContraSSFREX /17  /17  866087       04 0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS']

test_regex_combine()

In [16]:
testingdata2 = pure_data_lines[3483:]
test1 = combine_rows(testingdata2)
test1


['CERS     SS      NEW          200   2.350000    2.340000   2.35000006/1406/191706149900006 15:59:CustSS /17  /17  843318       57 0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS 854300',
 'CERS     SS      NEW          100   2.350000    2.340000   2.35000006/1406/191706149900006 15:59:ContraSSDTTX /17  /17  843340       58 0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS 854300',
 'CERS     SS      NEW          200   2.350000    2.340000   2.35000006/1406/191706149900006 15:59:CustSS /17  /17  843708       58 0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS 854300',
 'CERS     SS      NEW          786   2.350000    2.340000   2.35000006/1406/191706149900006 15:59:CustSS /17  /17  844088       59 0704 PerUnit   TERM  0.010000     AABBC     1706149900003#STA=CERS 854300']

In [86]:
sorted1 = regex_combine(pure_data_lines)
len(sorted1)

913

In [102]:
# need to find the headers and put them back in as the first row of the file
headers = example[4] + '    ' + example[6]
type(headers)

str

In [111]:
almost_there = '\n'.join(sorted1)
# almost_there (inspecting the file)
output = headers + '\n' + almost_there
# (inspecting output content before I write it)
# output[1000]
# len(output)
# output[191000:]


't   TERM  0.010000     AABBC     1706149900003#STA=CERS 854300'

In [112]:
# now it is time to write to a file
with open('sell_short_trades_2.txt', 'w') as outfile:
    outfile.write(output)